## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [3]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(tf.random.normal([784, 128]), name='W1')
        self.b1 = tf.Variable(tf.zeros([128]), name='b1')
        self.W2 = tf.Variable(tf.random.normal([128, 10]), name='W2')
        self.b2 = tf.Variable(tf.zeros([10]), name='b2')
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, [-1, 784])
        h1 = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [4]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [10]:
train_data, test_data = mnist_dataset()
# print(train_data[0].shape)
# for i in range(20):
#     print(train_data[1][i])
    
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 13.599497 ; accuracy 0.61083335
epoch 1 : loss 13.554316 ; accuracy 0.6117
epoch 2 : loss 13.509531 ; accuracy 0.61261666
epoch 3 : loss 13.4651375 ; accuracy 0.61336666
epoch 4 : loss 13.421129 ; accuracy 0.61405
epoch 5 : loss 13.377498 ; accuracy 0.61466664
epoch 6 : loss 13.334244 ; accuracy 0.61548334
epoch 7 : loss 13.291368 ; accuracy 0.61635
epoch 8 : loss 13.248864 ; accuracy 0.6170667
epoch 9 : loss 13.2067375 ; accuracy 0.61785
epoch 10 : loss 13.164965 ; accuracy 0.61861664
epoch 11 : loss 13.123551 ; accuracy 0.61931664
epoch 12 : loss 13.08249 ; accuracy 0.62025
epoch 13 : loss 13.041777 ; accuracy 0.62115
epoch 14 : loss 13.001402 ; accuracy 0.62186664
epoch 15 : loss 12.961368 ; accuracy 0.62238336
epoch 16 : loss 12.921668 ; accuracy 0.62298334
epoch 17 : loss 12.8823 ; accuracy 0.62365
epoch 18 : loss 12.843252 ; accuracy 0.6244
epoch 19 : loss 12.804519 ; accuracy 0.6253833
epoch 20 : loss 12.766102 ; accuracy 0.62585
epoch 21 : loss 12.727996 ; accura